## Xente Solution

In this notebook I have somethings I want to try out.

- The keras-pandas library: this is a library that helps in data preparation, the data once done, its ready to be used in a normal Machine Learning algorith or it can be used in a neural net.

I want to test this today and I see how it stacks up on the leader board, that is the score on the neural net.

In [1]:
#lets import the modules we are going to work with
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras import Sequential
from keras.layers import Dense

from keras_pandas import lib
from keras_pandas.Automater import Automater
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
#let's read in the data
train = pd.read_csv('training.csv',parse_dates=['TransactionStartTime'])
test = pd.read_csv('test.csv',parse_dates=['TransactionStartTime'])

In [3]:
#let see the data we are looking at, specifically the data types
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 16 columns):
TransactionId           95662 non-null object
BatchId                 95662 non-null object
AccountId               95662 non-null object
SubscriptionId          95662 non-null object
CustomerId              95662 non-null object
CurrencyCode            95662 non-null object
CountryCode             95662 non-null int64
ProviderId              95662 non-null object
ProductId               95662 non-null object
ProductCategory         95662 non-null object
ChannelId               95662 non-null object
Amount                  95662 non-null float64
Value                   95662 non-null int64
TransactionStartTime    95662 non-null datetime64[ns]
PricingStrategy         95662 non-null int64
FraudResult             95662 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(4), object(10)
memory usage: 11.7+ MB


In [4]:
#we have some object variables and numerical
#how big is our dataset
train.shape, test.shape

((95662, 16), (45019, 15))

In [5]:
#there are some columns we don't need ie transaction id for both datasets
fin_id = test['TransactionId'].values
test.drop(columns=['TransactionId','CurrencyCode','CountryCode','Amount'],inplace=True)
train.drop(columns=['TransactionId','CurrencyCode','CountryCode','Amount'],inplace=True)

In [6]:
ob_col = [col for col in train.columns if train[col].dtype=='object']
num_col = [col for col in train.columns if train[col].dtype!='object']

In [7]:
num_col

['Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult']

In [8]:
# TODO Train /test split
x_train_, x_test_ = train_test_split(train)

In [9]:
x_train_.head()

,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Value,TransactionStartTime,PricingStrategy,FraudResult
41492,BatchId_96402,AccountId_4269,SubscriptionId_1764,CustomerId_4722,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000,2018-12-26 18:45:43,2,0
4676,BatchId_97058,AccountId_1591,SubscriptionId_1570,CustomerId_1969,ProviderId_6,ProductId_3,airtime,ChannelId_3,1000,2018-11-20 18:11:54,2,0
92856,BatchId_20780,AccountId_3562,SubscriptionId_3768,CustomerId_4000,ProviderId_3,ProductId_3,airtime,ChannelId_3,2000,2019-02-10 18:34:25,1,0
35109,BatchId_125233,AccountId_1115,SubscriptionId_4264,CustomerId_1472,ProviderId_1,ProductId_15,financial_services,ChannelId_3,5000,2018-12-21 09:44:43,2,0
20345,BatchId_135431,AccountId_4841,SubscriptionId_3829,CustomerId_4518,ProviderId_4,ProductId_6,financial_services,ChannelId_2,50,2018-12-07 18:01:28,2,0


In [10]:
# TODO List out variable types

data_type_dict = {'numerical': num_col,
                      'categorical': ob_col,
                    }
output_var = 'FraudResult'

In [11]:
# Create and fit Automater
# This is the best part of this module, here it prepares the data for any machine learning model, neural nets
# or other models.
# let's see how it does that now
auto = Automater(data_type_dict=data_type_dict, output_var=output_var)
auto.fit(train)

In [12]:
#let's now use the model to transform our dataset that we will use in the machine learning
train_X, train_y = auto.fit_transform(x_train_)
test_X, test_y = auto.transform(x_test_)

In [13]:
#right now we should be having data that is already pre-formatted to be used in our models
pd.DataFrame(train_X).T.shape,pd.DataFrame(train_y).T.shape

((71746, 11), (1, 71746))

In [14]:
pd.DataFrame(train_y).T.shape

(1, 71746)

In [15]:
pd.Series(train_y).T.shape

(71746,)

## Test Run

Let's take this bad boy for a test run with a neural network.
We are going to keep it simple so we can see how it does, in terms of accuracy.

In [39]:
#lets try this in a simple neural net

from keras import Model
from keras.layers import Dense
from sklearn.model_selection import train_test_split

x = auto.input_nub
x = Dense(128)(x)
x = Dense(64)(x)
x = Dense(32)(x)
x = Dense(32)(x)
x = auto.output_nub(x)

In [44]:
model = Model(inputs=auto.input_layers, outputs=x)
model.compile(optimizer='adam', loss=auto.suggest_loss())

In [ ]:
model.fit(train_X, train_y,epochs=10)

Epoch 1/10
71746/71746 [==============================] - 151s 2ms/step - loss: 0.4436
Epoch 2/10
71746/71746 [==============================] - 145s 2ms/step - loss: 0.4277
Epoch 3/10
71746/71746 [==============================] - 147s 2ms/step - loss: 0.3661
Epoch 4/10
68032/71746 [===========================>..] - ETA: 7s - loss: 0.3123

In [27]:
model.evaluate(test_X, test_y)

23916/23916 [==============================] - 1s 31us/step


0.9895799511610255

In [ ]:
pred_test_y = model.predict(test_X)
auto.inverse_transform_output(pred_test_y)

### Test Data Set

In [ ]:
# TODO List out variable types

data_type_dict_ = {'numerical': ['Value', 'TransactionStartTime', 'PricingStrategy'],
                      'categorical': ob_col,
                    }
output_var = None

In [ ]:
auto_ = Automater(data_type_dict=data_type_dict_, output_var=output_var)
test = auto_.fit_transform(test)

In [ ]:
#let's now build random base models and check how they are doing

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.svm import NuSVC,SVC
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier

from xgboost import XGBClassifier

In [ ]:
#let's initialize the modles here
sgd = SGDClassifier()
rdm = RadiusNeighborsClassifier()
ada = AdaBoostClassifier()
grd = GradientBoostingClassifier()
nsv = NuSVC()
svc = SVC()
knn = KNeighborsClassifier()
rnc = RadiusNeighborsClassifier()
xgb = XGBClassifier()
rdmf = RandomForestClassifier

In [ ]:
rdmf.fit(train_X,train_y)

In [ ]:
models = {
'SGDClassifier':sgd,
'RadiusNeighborsClassifier':rdm,
'AdaBoostClassifier':ada,
'GradientBoostingClassifier':grd,
'NuSVC':nsv,
'SVC':svc,
'KNeighborsClassifier':knn,
'RadiusNeighborsClassifier':rnc,
'XGBClassifier':xgb
}

I am having a hard time in converting the data that I have into a fomat that can be accepted by the models above, We have managed to make it work with the neural network.

In [ ]:
#now that we have the a data structure let's loop through the structure and fit our data
from sklearn.metrics import f1_score

#result dataframe
result = pd.DataFrame()

for model_name, model in models.items():
    #let's fit the model
    model.fit(pd.DataFrame(train_X).T, pd.Series(train_y).T)
    
    #let's pred on the test data
    pred = model.predict(pd.DataFrame(test_X).T)
    
    #score the 
    score = f1_score(pd.Series(test_y),pred)
    
    #lets add this to a dataframe
    result[model_name]=score

## What have I learnt:

1- We may need to do the initial data analysis but not as intensely as we used to before. Because in the end I need to know which columns are going to fall off or what

2- We need to be able to handle data in different dimensions and data types, ndarrays, dataframes, dictionaries and others, because when it comes to the time of fitting this data into normal models it takes accuracy in those areas for you not to waste time.